In [ ]:
import pandas as pd
pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(color_codes=True)
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import re
import nltk
import string
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
wchat=pd.read_csv("C:/Users/bansa/Downloads/WhatsApp Chat - Ashraf LBSIM/_chat.txt", delimiter="\t",header=None)

In [ ]:
wchat.head()

In [ ]:
wchat.drop(index=[0], inplace =True)

In [ ]:
wchat.reset_index(drop=True, inplace=True)

In [ ]:
wchat.rename(columns={0:"Chat"}, inplace=True)

In [ ]:
wchat

In [ ]:
wchat[['Date', 'Chat_updated']] = wchat['Chat'].str.split(']', expand=True)

In [ ]:
wchat

In [ ]:
# Checking the distribution of length of chats
plt.figure(figsize=(10,5))
plt.hist(wchat["Chat_updated"].str.len(), bins=20)
plt.show()

### Text Preprocessing

In [ ]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt

In [ ]:
wchat["Chat_updated"] = wchat["Chat_updated"].str.replace("rubal:","")
wchat["Chat_updated"] = wchat["Chat_updated"].str.replace("Ashraf LBSIM: ","")

In [ ]:
#Removing Punctuations, Numbers, and Special Characters
#we will replace everything except characters and hashtags with spaces
#The regular expression “[^a-zA-Z#]” means anything except alphabets and ‘#’.
wchat.Chat_updated = wchat.Chat_updated.str.replace("[^a-zA-Z]", " ")
wchat.head(10)

In [ ]:
wchat.dtypes

In [ ]:
wchat.Chat_updated[0]

In [ ]:
len(wchat)

In [ ]:
# Removing words with length less than or equal to 3
def remove_short_words(x):
    for i in range(len(wchat)):
        if x[i] is not None:
            x[i]=' '.join([w for w in x[i].split() if len(w) > 3])
    return x

In [ ]:
wchat.Chat_updated=remove_short_words(wchat.Chat_updated)

In [ ]:
# Removing any extra spaces
def remove_blancs(x):
    for i in range(len(x)):
        if x[i] is not None:
            x[i]=' '.join([w.strip() for w in x[i].split()])
    return x

In [ ]:
wchat.Chat_updated=remove_blancs(wchat.Chat_updated)

In [ ]:
# Removing stopwords
stopwords=nltk.corpus.stopwords.words("English")

In [ ]:
def remove_stopwords(x):
    for i in range(len(x)):
        if x[i] is not None:
            new_ls=[]
            x_spl= x[i].split()
            for w in x_spl:
                if w not in stopwords:
                    new_ls.append(w)
                else:
                    new_ls.append(" ")
            x[i]=" ".join(new_ls)
    return x

In [ ]:
wchat.Chat_updated=remove_stopwords(wchat.Chat_updated)

In [ ]:
wchat

### Text Normalization

In [ ]:
nltk.download("wordnet")

In [ ]:
lemma=WordNetLemmatizer()

In [ ]:
i=0
for chat in wchat.Chat_updated:
    chat=str(chat).split()
    chat=[lemma.lemmatize(word,pos="v") for word in chat]
    wchat.Chat_updated[i]=chat
    i+=1

In [ ]:
# Lets stitch them together
j=0
for chat in wchat.Chat_updated:
    wchat.Chat_updated[j]=" ".join([word for word in chat])
    j+=1

In [ ]:
wchat

In [ ]:
### Removing "image omit" pattern from chats
for i in range(len(wchat.Chat_updated)):
    wchat.Chat_updated[i]=remove_pattern(wchat.Chat_updated[i],"image omit")

In [ ]:
wchat

### Creating and Saving Word Clouds

In [ ]:
all_words=" ".join([text for text in wchat.Chat_updated])
wordcloud=WordCloud(width=700, height=800, random_state=200,max_font_size=110).generate(all_words)
plt.figure(figsize=(15,15))
plt.imshow(wordcloud,interpolation="bilinear")
plt.axis("off")
wordcloud.to_file("I:/Data Science/Eduonix/Assignments/16267625/Whatsapp_GroupChat_Analysis/wordcloud.png")
plt.show()

### Sentiment Analysis

In [ ]:
wchat.head()

In [ ]:
#nltk.download('vader_lexicon')

In [ ]:
sentiments = SentimentIntensityAnalyzer()
wchat["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in wchat["Chat_updated"]]
wchat["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in wchat["Chat_updated"]]
wchat["Neutral"] = [sentiments.polarity_scores(i)["neu"] for i in wchat["Chat_updated"]]

In [ ]:
wchat

In [ ]:
a=sum(wchat.Positive)
b=sum(wchat.Negative)
c=sum(wchat.Neutral)

In [ ]:
print(a,b,c)

In [ ]:
def sentiment_analysis(x,y,z):
    if (x>y) and (x>z):
        print("Positive")
    elif (y>x) and (y>z):
        print("Negative")
    else:
        print("Neutral")

In [ ]:
sentiment_analysis(a,b,c)